- Mạng nơ-ron tế bào bậc hai có cấu trúc tương tự như bậc nhất, chỉ là thay vì có 1 ma trận điều khiển B, 1 ma trận phản hồi A và mức ngưỡng I như mạng Nơ-ron tế bào bậc nhất thì bậc hai có 10 ma trận điều khiến (gồm 1 ma trận B1 và 9 ma trận lần lượt là B21 đến B29), 10 ma trận phản hồi (gồm 1 ma trận A1 và 9 ma trận A21 đến A29) và mức ngưỡng I.
- Do các ma trận đều đối xứng để đảm bảo tính ổn định cho mạng, nên quần thể sẽ gồm (1+9+1+9)*5+1=101 trọng số.

In [1]:
from scipy.signal import convolve2d
from scipy.integrate import ode
from PIL import Image as img
import numpy as np
import numpy.matlib as mat
import math
from PIL import Image as img

In [ ]:
# hàm trạng thái
def hamTrangThai(t, x, u, B1, B2, A1, A2, I, n, m):
    x = x.reshape(n, m)
    dx = -x + convolve2d(u, B, 'same') + convolve2d(hamDauRa(x), A, 'same') + I
    return dx.reshape(n*m)

# hàm đầu ra
def hamDauRa(x):
    return 0.5 * (abs(x + 1) - abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    #arr2[0], arr2[2] = arr2[2], arr2[0]#nếu có thì sẽ đối xứng theo đường chéo phụ, nếu không thì sẽ là đối xứng tâm
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Hàm lai hai số
def lai(a,b):#k là điểm lai  -3,|45
    if(a>0):
        return math.modf(a)[1]+abs(math.modf(b)[0])
    return math.modf(a)[1]-abs(math.modf(b)[0])

# Hàm tạo số ngẫu nhiên từ -9.99 đến 9.99
def ngauNhien():
    return np.random.randint(-999,1000)/100.

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1 # h = 3

class CNN:

    slts = 11 # số lượng trọng số trong quần thể

    # Quần thể trọng số (gồm 11 số)
    #quanThe = np.array([ngauNhien() for i in range(slts)])
    quanThe = np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,2.,-1.])

    # ma trận ảnh đầu vào u
    # u = np.array([
    #     [-1,-1,-1,-1,-1, 1, 1, 1],
    #     [-1, 1, 1, 1,-1, 1, 1, 1],
    #     [-1, 1, 1, 1,-1, 1, 1, 1],
    #     [-1, 1, 1, 1,-1,-1,-1,-1],
    #     [-1, 1, 1, 1, 1, 1,-1,-1],
    #     [-1, 1, 1, 1, 1, 1,-1,-1],
    #     [-1, 1, 1, 1, 1, 1,-1,-1],
    #     [-1,-1,-1,-1,-1,-1,-1,-1]], dtype=float)*-1.0
    u = np.array(img.open('images/u2.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

    # kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
    m, n = u.shape
    print(m,n)

    # ma trận đầu ra mong muốn d
    # d = np.array([
    #     [-1,-1,-1,-1,-1, 1, 1, 1],
    #     [-1, 1, 1, 1,-1, 1,-1, 1],
    #     [-1, 1,-1, 1,-1, 1, 1, 1],
    #     [-1, 1,-1, 1,-1,-1,-1,-1],
    #     [-1, 1,-1, 1, 1, 1,-1,-1],
    #     [-1, 1,-1,-1,-1, 1,-1,-1],
    #     [-1, 1, 1, 1, 1, 1,-1,-1],
    #     [-1,-1,-1,-1,-1,-1,-1,-1]], dtype=float)*-1.0
    d = np.array(img.open('images/d2.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

    # ma trận điều khiển B ban đầu
    # B = np.array([
    #     [-1.,-1.,-1.],
    #     [-1., 8.,-1.],
    #     [-1.,-1.,-1.]])
    B = taoMaTranDoiXung(quanThe[:5])

    # ma trận phản hồi A ban đầu
    # A = np.array([
    #     [ 0., 0., 0.],
    #     [ 0., 1., 0.],
    #     [ 0., 0., 0.]])
    A = taoMaTranDoiXung(quanThe[5:10])

    # mức ngưỡng I ban đầu
    I = quanThe[10]
    #I = -1.0

    # hai giá trị ban đầu cho ptvp gồm x0 = [ma trận toàn số 0] với t0 = 0.0 và t1 để tính đầu ra x1 (t1 != t0)
    x0, t0, t1 = u*0.0, 0.0, 1.0

    # ma trận đầu ra y (y được tính theo x)
    y = hamDauRa(x0)

    # sai số cho phép
    E0 = 4.0

    # sai số hiện tại
    E = float('inf') # mặc định ban đầu gán với dương vô cùng

cnn = CNN()